In [1]:
import sys

In [2]:
sys.path.append("/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from pydub import AudioSegment
from pydub.playback import play
import librosa
import librosa.display

from IPython.display import Audio
from numpy.fft import fft, ifft
import sys
%matplotlib inline

## PyDub ##

In [4]:
#read in song
clime_pt_2 = AudioSegment.from_wav('Audio_Samples/10 Clime Part Two.L.wav')

In [5]:
#get duration
clime_pt_2.duration_seconds

505.32

In [6]:
#slice audio
seconds = 1000

favorite_part = clime_pt_2[285*seconds:290*seconds]

In [7]:
#playback
play(favorite_part)

Could not import the PyAudio C module '_portaudio'.


In [8]:
#reverse, for some lolz
favorite_reversed = favorite_part.reverse()
play(favorite_reversed)

Could not import the PyAudio C module '_portaudio'.


In [9]:
#crossfade for fun
favorite_faded = favorite_part.fade_in(2*seconds).fade_out(2*seconds)
play(favorite_faded)

Could not import the PyAudio C module '_portaudio'.


In [10]:
#convert to array of samples
sound = clime_pt_2.set_frame_rate(16000)
channel_sounds = sound.split_to_mono()
samples = [s.get_array_of_samples() for s in channel_sounds]

fp_arr = np.array(samples).T.astype(np.float32)

In [11]:
len(fp_arr)

8085120

In [12]:
fp_arr

array([[-3.],
       [-3.],
       [-3.],
       ...,
       [ 0.],
       [ 0.],
       [ 0.]], dtype=float32)

## Librosa ##

In [13]:
#load in audio
clime_pt_2_path = 'Audio_Samples/10 Clime Part Two.L.wav'
x , sr = librosa.load(clime_pt_2_path)
print(type(x), type(sr))

<class 'numpy.ndarray'> <class 'int'>


In [ ]:
#we can also plot waveform in librosa
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

In [ ]:
#zero crossing for a segment, used heavily in speech recognition and MIR, associated with percussive musics
n0 = 9000
n1 = 9100
plt.figure(figsize=(14, 5))
plt.plot(x[n0:n1])
plt.grid()

In [ ]:
# calculate zero crossings manually
zero_crossings = librosa.zero_crossings(x[n0:n1], pad=False)
print(sum(zero_crossings))

In [ ]:
#or a spectrogram, showing presence of frequenices
X = librosa.stft(x) #fourier transform
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
plt.colorbar()

In [ ]:
#log
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

In [ ]:
#estimate tempo for song
onset_env = librosa.onset.onset_strength(x, sr=sr)
tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
tempo

In [ ]:
#extract harmonic and percussive data
harmonic, percussive = librosa.effects.hpss(x)

#plot pitch classes
chroma = librosa.feature.chroma_cqt(y=harmonic, sr=sr)
plt.figure(figsize=(18,5))
librosa.display.specshow(chroma, sr=sr, x_axis='time', y_axis='chroma', vmin=0, vmax=1)
plt.title('Chromagram') 
plt.colorbar();